In [1]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import contextily as ctx
import copy
import math
from scipy.stats import norm
from pathlib import Path

from pyincore import (
    IncoreClient,
    Dataset,
    DataService,
    HazardService,
    FragilityService,
    MappingSet,
    FragilityCurveSet,
)
from pyincore import HHRSOutputProcess
from pyincore.utils.dataprocessutil import DataProcessUtil


# importing pyIncone analyses:
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.nonstructbuildingdamage import NonStructBuildingDamage
from pyincore.analyses.buildingfunctionality import BuildingFunctionality
from pyincore.analyses.combinedwindwavesurgebuildingdamage import (
    CombinedWindWaveSurgeBuildingDamage,
)
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability
from pyincore.analyses.housingunitallocation import HousingUnitAllocation
from pyincore.analyses.populationdislocation import (
    PopulationDislocation,
    PopulationDislocationUtil,
)
from pyincore.analyses.housingrecoverysequential import HousingRecoverySequential
from pyincore.analyses.socialvulnerability import SocialVulnerability
from pyincore.analyses.capitalshocks import CapitalShocks
from pyincore.analyses.galvestoncge import GalvestonCGEModel
import json

In [2]:
client = IncoreClient()
# IN-CORE caches files on the local machine, it might be necessary to clear the memory
# client.clear_cache()
data_service = DataService(client)  # create data_service object for loading files
hazard_service = HazardService(client)
fragility_services = FragilityService(client)

Connection successful to IN-CORE services. pyIncore version detected: 1.20.1


## Combined Building Damage


### Wind building damage for Galveston, TX using Hurricane Ike


In [3]:
hazard_type = "hurricane"

# Galveston Hurricane Ike
hazard_id = "5fa5a228b6429615aeea4410"

# Galveston inventory data
# bldg_dataset_id = "63053ddaf5438e1f8c517fed" # Full city
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"  # Island Only

# Building Wind Fragility mapping
mapping_id = "62fef3a6cef2881193f2261d"
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [4]:
# wind building damage
w_bldg_dmg = BuildingDamage(client)
w_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
w_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
w_bldg_dmg.set_parameter("result_name", "Galveston-wind-dmg")
w_bldg_dmg.set_parameter("hazard_type", hazard_type)
w_bldg_dmg.set_parameter("hazard_id", hazard_id)
w_bldg_dmg.set_parameter("num_cpu", 8)
w_bldg_dmg.run_analysis()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

### Surge-Wave building damage for Galveston, TX using Hurricane Ike


In [5]:
# Surge-wave mapping
mapping_id = "6303e51bd76c6d0e1f6be080"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [6]:
# surge-wave building damage
sw_bldg_dmg = BuildingDamage(client)
sw_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
sw_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
sw_bldg_dmg.set_parameter("result_name", "Galveston-sw-dmg")
sw_bldg_dmg.set_parameter("hazard_type", hazard_type)
sw_bldg_dmg.set_parameter("hazard_id", hazard_id)
sw_bldg_dmg.set_parameter("num_cpu", 8)
sw_bldg_dmg.run_analysis()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

### Flood building damage for Galveston, TX using Hurricane Ike


In [7]:
# flood mapping
mapping_id = "62fefd688a30d30dac57bbd7"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [8]:
# flood building damage
f_bldg_dmg = NonStructBuildingDamage(client)
f_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
f_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
f_bldg_dmg.set_parameter("result_name", "Galveston-flood-dmg")
f_bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
f_bldg_dmg.set_parameter("hazard_type", hazard_type)
f_bldg_dmg.set_parameter("hazard_id", hazard_id)
f_bldg_dmg.set_parameter("num_cpu", 8)
f_bldg_dmg.run_analysis()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

### Combine wind, wave and surge building damage


In [9]:
surge_wave_damage = sw_bldg_dmg.get_output_dataset("ds_result")
wind_damage = w_bldg_dmg.get_output_dataset("ds_result")
flood_damage = f_bldg_dmg.get_output_dataset("result")

In [10]:
combined_bldg_dmg = CombinedWindWaveSurgeBuildingDamage(client)
result_name = "Galveston-combined-dmg"
combined_bldg_dmg.set_input_dataset("surge_wave_damage", surge_wave_damage)
combined_bldg_dmg.set_input_dataset("wind_damage", wind_damage)
combined_bldg_dmg.set_input_dataset("flood_damage", flood_damage)
combined_bldg_dmg.set_parameter("result_name", result_name)
combined_bldg_dmg.run_analysis()

True

In [11]:
combined_dmg = combined_bldg_dmg.get_output_dataset("ds_result")
combined_dmg_df = combined_dmg.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
combined_dmg_df.head()

,guid,LS_0,LS_1,LS_2,DS_0,DS_1,DS_2,DS_3,haz_expose
0,f3a21b5f-0334-4d77-911f-faa9a45c92d7,0.000000e+00,0.0,0.0,1.000000,0.000000e+00,0.0,0.0,yes
1,80ab0289-8c72-4699-b12b-08ef2f2a5c91,7.000482e-04,0.0,0.0,0.999300,7.000482e-04,0.0,0.0,yes
2,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,1.544290e-05,0.0,0.0,0.999985,1.544290e-05,0.0,0.0,yes
3,79008971-a534-4e45-9e40-43a2a904e59b,3.000000e-10,0.0,0.0,1.000000,3.000000e-10,0.0,0.0,yes
4,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,6.000000e-10,0.0,0.0,1.000000,6.000000e-10,0.0,0.0,yes


In [12]:
combined_dmg_rs = combined_bldg_dmg.get_output_dataset("result")
combined_dmg_df_rs = combined_dmg_rs.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
combined_dmg_df_rs

,guid,w_maxprob,w_max_ds,sw_maxprob,sw_max_ds,f_maxprob,f_max_ds,max_state
0,f3a21b5f-0334-4d77-911f-faa9a45c92d7,1.000000,DS_0,NaN,NaN,0.990352,DS_0,DS_0
1,80ab0289-8c72-4699-b12b-08ef2f2a5c91,0.999300,DS_0,NaN,NaN,0.479312,DS_2,DS_2
2,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,0.999985,DS_0,NaN,NaN,0.557246,DS_1,DS_1
3,79008971-a534-4e45-9e40-43a2a904e59b,1.000000,DS_0,NaN,NaN,0.584024,DS_1,DS_1
4,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,1.000000,DS_0,NaN,NaN,0.999695,DS_0,DS_0
...,...,...,...,...,...,...,...,...
29536,1ac5be10-582c-46b2-91ed-469f684bb886,0.987090,DS_0,1.000000,DS_0,0.675828,DS_1,DS_1
29537,fbcfb6d6-de13-42fa-9839-d7c80afc651d,0.999735,DS_0,1.000000,DS_0,1.000000,DS_0,DS_0
29538,2ee961c9-cd94-499e-b1fe-3e0220a440ea,0.999790,DS_0,0.999998,DS_0,0.820926,DS_1,DS_1
29539,fa15cfbc-8a60-4a3f-b40f-7a6fada625b9,1.000000,DS_0,NaN,NaN,0.571071,DS_1,DS_1


### Archetype Mapping


In [13]:
buildings = Dataset.from_data_service(bldg_dataset_id, data_service)
bldg_gdf = buildings.get_dataframe_from_shapefile()
bldg_gdf.head()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,strctid,parid,struct_typ,year_built,no_stories,a_stories,b_stories,bsmt_type,sq_foot,gsq_foot,...,ffe_elev,g_elev,archetype,arch_wind,arch_flood,arch_sw,csv_guid,csv_sector,csv_val_st,geometry
0,STf3a21b5f-0334-4d77-911f-faa9a45c92d7,0,None,1980,2,0,0,0,47927,0,...,3.76991,3.46511,0,15,6,15,None,None,None,POINT (-94.81680 29.31409)
1,ST80ab0289-8c72-4699-b12b-08ef2f2a5c91,0,None,1954,1,0,0,1,100796,0,...,2.26048,1.95568,0,15,6,15,None,None,None,POINT (-94.82295 29.29635)
2,ST7916eaa8-0bfa-4b32-84ff-20f19a520f5c,0,None,1970,1,0,0,1,27121,0,...,2.68011,2.37531,0,15,6,15,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,IRetail,2435054.5,POINT (-94.82423 29.29098)
3,ST79008971-a534-4e45-9e40-43a2a904e59b,0,None,2002,1,0,0,1,126333,0,...,2.91465,2.60985,0,15,6,15,79008971-a534-4e45-9e40-43a2a904e59b,IProfSer,4483599,POINT (-94.84045 29.29117)
4,ST2ad8ab6f-057f-468e-9911-6e92d9d4d01f,0,None,2002,1,0,0,1,114544,0,...,3.73770,3.43290,0,15,6,15,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,IRealE,282702.06,POINT (-94.84242 29.29059)


In [14]:
arch_mapping = Dataset.from_data_service(
    "63dc1f1362b9d001e6a1b485", data_service
).get_dataframe_from_csv()
arch_mapping

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,arch_wind,category,cluster
0,12.0,Critical Facilities,Critical Medical - Acute Care Hospitals
1,14.0,Critical Facilities,Emergency Operations Centers
2,11.0,Critical Facilities,Critical Government - First Responder Facilities
3,NaN,Critical Facilities,"Non-ambulatory Facilities - Prisons, nursing h..."
4,NaN,Emergency Housing,Emergency Shelters
5,NaN,Emergency Housing,Residential Housing
6,NaN,Emergency Housing,SFH and Multi-family
7,NaN,Housing / Neighborhood,Critical Retail
8,13.0,Housing / Neighborhood,Religious and Spiritual Centers
9,1.0,Housing / Neighborhood,Residential Housing


In [15]:
max_dmg_state_df = DataProcessUtil.get_max_damage_state(combined_dmg_df)
max_dmg_state_df

,guid,max_prob,max_state
0,f3a21b5f-0334-4d77-911f-faa9a45c92d7,1.000000,DS_0
1,80ab0289-8c72-4699-b12b-08ef2f2a5c91,0.999300,DS_0
2,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,0.999985,DS_0
3,79008971-a534-4e45-9e40-43a2a904e59b,1.000000,DS_0
4,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,1.000000,DS_0
...,...,...,...
29536,1ac5be10-582c-46b2-91ed-469f684bb886,0.987090,DS_0
29537,fbcfb6d6-de13-42fa-9839-d7c80afc651d,0.999735,DS_0
29538,2ee961c9-cd94-499e-b1fe-3e0220a440ea,0.820926,DS_1
29539,fa15cfbc-8a60-4a3f-b40f-7a6fada625b9,1.000000,DS_0


In [16]:
ret_json = DataProcessUtil.create_mapped_dmg_result_gal(
    bldg_gdf, combined_dmg_df_rs, arch_mapping, "max_state", "arch_wind"
)
ret_json

{'by_cluster': [{'category': 'Critical Facilities',
   'cluster': 'Critical Medical - Acute Care Hospitals',
   'Affected': 15,
   'Destroyed': 0,
   'Major': 0,
   'Minor': 0},
  {'category': 'Critical Facilities',
   'cluster': 'Emergency Operations Centers',
   'Affected': -1,
   'Destroyed': -1,
   'Major': -1,
   'Minor': -1},
  {'category': 'Critical Facilities',
   'cluster': 'Critical Government - First Responder Facilities',
   'Affected': -1,
   'Destroyed': -1,
   'Major': -1,
   'Minor': -1},
  {'category': 'Critical Facilities',
   'cluster': 'Non-ambulatory Facilities - Prisons, nursing homes, etc.',
   'Affected': -1,
   'Destroyed': -1,
   'Major': -1,
   'Minor': -1},
  {'category': 'Emergency Housing ',
   'cluster': 'Emergency Shelters',
   'Affected': -1,
   'Destroyed': -1,
   'Major': -1,
   'Minor': -1},
  {'category': 'Emergency Housing ',
   'cluster': 'Residential Housing',
   'Affected': -1,
   'Destroyed': -1,
   'Major': -1,
   'Minor': -1},
  {'category': 

## Electric Power Pole Damage


In [17]:
# EPF fragility mapping
epf_mapping_id = "62fac92ecef2881193f22613"
epf_mapping_set = MappingSet(fragility_service.get_mapping(epf_mapping_id))

epf_dmg_hurricane_galveston = EpfDamage(client)
epf_dmg_hurricane_galveston.load_remote_input_dataset(
    "epfs", "62fc000f88470b319561b58d"
)
epf_dmg_hurricane_galveston.set_input_dataset("dfr3_mapping_set", epf_mapping_set)
epf_dmg_hurricane_galveston.set_parameter(
    "result_name", "Galveston-hurricane-epf-damage"
)
epf_dmg_hurricane_galveston.set_parameter(
    "fragility_key", "Non-Retrofit Fragility ID Code"
)
epf_dmg_hurricane_galveston.set_parameter("hazard_type", hazard_type)
epf_dmg_hurricane_galveston.set_parameter("hazard_id", hazard_id)
epf_dmg_hurricane_galveston.set_parameter("num_cpu", 8)

# Run Analysis
epf_dmg_hurricane_galveston.run_analysis()
epf_dmg_result = epf_dmg_hurricane_galveston.get_output_dataset("result")

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


## Monte Carlo Failure Probability


In [18]:
mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", combined_dmg)
mc.set_parameter("result_name", "combined_bldg_mc")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 100)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc.set_parameter("failure_state_keys", ["DS_2", "DS_3", "DS_4"])
mc.run_analysis()

gal_bldg_damage_mcs_samples = mc.get_output_dataset("sample_failure_state")
gal_bldg_damage_mcs_samples_df = gal_bldg_damage_mcs_samples.get_dataframe_from_csv()
gal_bldg_damage_mcs_samples_df.head()

,guid,failure
0,f3a21b5f-0334-4d77-911f-faa9a45c92d7,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
1,80ab0289-8c72-4699-b12b-08ef2f2a5c91,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
2,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
3,79008971-a534-4e45-9e40-43a2a904e59b,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
4,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."


### Clustering


In [19]:
ret_json = DataProcessUtil.create_mapped_func_result(
    bldg_gdf, gal_bldg_damage_mcs_samples_df, arch_mapping, arch_col="arch_wind"
)
with open("galveston_mcs_cluster.json", "w") as f:
    json.dump(ret_json, f, indent=2)

In [20]:
gal_bldg_failure_probability = mc.get_output_dataset(
    "failure_probability"
)  # get buildings failure probabilities

In [21]:
mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", epf_dmg_result)
mc.set_parameter("result_name", "epf_mc")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 100)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc.set_parameter("failure_state_keys", ["DS_2", "DS_3", "DS_4"])
mc.run_analysis()

gal_pole_damage_mcs_samples = mc.get_output_dataset("sample_failure_state")
gal_pole_damage_mcs_samples_df = gal_pole_damage_mcs_samples.get_dataframe_from_csv()
gal_pole_damage_mcs_samples_df.head()

,guid,failure
0,2525e200-cdc6-4fb8-9952-f17579176fd8,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
1,9180f947-26ae-40bd-85f5-83a80525f6c8,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
2,11ddc2dc-cc92-4c90-b4cc-a99b92a26f4b,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
3,9e58f196-9a2c-40b1-b722-69fa9c53c9d4,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
4,a4de2d54-b251-4a02-af0c-d12e0dc3670c,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."


## Building Functionality


In [22]:
gal_bldg_func = BuildingFunctionality(client)

gal_bldg_func.set_input_dataset(
    "building_damage_mcs_samples", gal_bldg_damage_mcs_samples
)
gal_bldg_func.set_input_dataset("poles_damage_mcs_samples", gal_pole_damage_mcs_samples)
gal_bldg_func.load_remote_input_dataset(
    "interdependency_dictionary", "63e5af0262d82d2f5a1058ea"
)

gal_bldg_func.set_parameter("result_name", "gal_mcs_functionality_probability")
gal_bldg_func.run_analysis()

gal_bldg_func_samples_df = gal_bldg_func.get_output_dataset(
    "functionality_samples"
).get_dataframe_from_csv()
gal_bldg_func_samples_df.head()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,guid,failure
0,f3a21b5f-0334-4d77-911f-faa9a45c92d7,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
1,80ab0289-8c72-4699-b12b-08ef2f2a5c91,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
2,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
3,79008971-a534-4e45-9e40-43a2a904e59b,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
4,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."


In [23]:
ret_json = DataProcessUtil.create_mapped_func_result(
    bldg_gdf, gal_bldg_func_samples_df, arch_mapping, arch_col="arch_wind"
)
with open("galveston_bldg_func_cluster.json", "w") as f:
    json.dump(ret_json, f, indent=2)

# Capital Shocks


In [24]:
# Create Capital shocks analysis
capital_shocks = CapitalShocks(client)
building_to_sectors_id = "640200da475d895dfc2de1bd"
# Load remote datasets
capital_shocks.load_remote_input_dataset("buildings_to_sectors", building_to_sectors_id)
capital_shocks.load_remote_input_dataset("buildings", bldg_dataset_id)
capital_shocks.set_input_dataset("failure_probability", gal_bldg_failure_probability)

capital_shocks.set_parameter(
    "result_name", "sector_shocks"
)  # name of csv file with results

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [25]:
# Run capital shocks analysis
capital_shocks.run_analysis()

sector_shocks_result = capital_shocks.get_output_dataset("sector_shocks")
sector_shocks_result.get_dataframe_from_csv()

,sector,shock
0,HS3I,0.910664
1,HS2I,0.922722
2,HS1I,0.951555
3,IPROFSER,0.885691
4,IRETAIL,0.864577
5,IART,0.869340
6,ICONS,0.822254
7,IHEALTH,0.914509
8,ITRANS,0.761241
9,IREALE,0.932839


# CGE


In [26]:
galveston_cge = GalvestonCGEModel(client)

galveston_cge.set_parameter("model_iterations", 1)
galveston_cge.load_remote_input_dataset("SAM", "641e1fccea308903b2b8c51a")
galveston_cge.load_remote_input_dataset("BB", "641e1fff34810d74880b3985")
galveston_cge.load_remote_input_dataset("MISCH", "641e21a702414a24a5436dfe")
galveston_cge.load_remote_input_dataset("EMPLOY", "641e2152ea308903b2b8c51c")
galveston_cge.load_remote_input_dataset("JOBCR", "641e209eea308903b2b8c51b")
galveston_cge.load_remote_input_dataset("OUTCR", "641e20f134810d74880b3986")
galveston_cge.set_input_dataset("sector_shocks", sector_shocks_result)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [27]:
galveston_cge.run_analysis()

Calibration: 
CPIEQ(H)
0 0 0.0
0 1 0.0
0 2 0.0
0 3 0.0
0 4 0.0
0 5 0.0
0 6 0.0
0 7 0.0
0 8 0.0
0 9 0.0
None
YEQ(H)
0 0 0.0
1 0 0.0
2 0 0.0
3 0 -2.2737367544323206e-13
4 0 0.0
5 0 0.0
6 0 2.842170943040401e-14
7 0 1.1368683772161603e-13
8 0 0.0
9 0 4.547473508864641e-13
None
YDEQ(H)
0 0 1.420147555108997e-09
1 0 3.2536320304643596e-09
2 0 -4.603180059348233e-10
3 0 -3.345940058352426e-08
4 0 -5.479751052916981e-08
5 0 6.85554368828889e-09
6 0 1.512785274826456e-08
7 0 -2.839033186319284e-08
8 0 -2.4542896426282823e-07
9 0 -3.193035809090361e-07
None
CHEQ(I,H)
0 0 0.0
0 1 0.0
0 2 0.0
0 3 0.0
0 4 0.0
0 5 -0.0
0 6 -0.0
0 7 -0.0
0 8 -0.0
0 9 -0.0
1 0 0.0
1 1 0.0
1 2 0.0
1 3 0.0
1 4 0.0
1 5 -0.0
1 6 -0.0
1 7 -0.0
1 8 -0.0
1 9 -0.0
2 0 -0.0
2 1 -0.0
2 2 -0.0
2 3 -0.0
2 4 -0.0
2 5 -0.0
2 6 -0.0
2 7 -0.0
2 8 -0.0
2 9 -0.0
3 0 0.0
3 1 0.0
3 2 0.0
3 3 0.0
3 4 0.0
3 5 -0.0
3 6 -0.0
3 7 -0.0
3 8 -0.0
3 9 -0.0
4 0 0.0
4 1 0.0
4 2 0.0
4 3 0.0
4 4 0.0
4 5 -0.0
4 6 -0.0
4 7 -0.0
4 8 -0.0
4 9 -0.0
5 0 0

## Population Dislocation


In [28]:
pop_dis = PopulationDislocation(client)
value_loss = "60354810e379f22e16560dbd"
bg_data = "603545f2dcda03378087e708"
# hua_result_id = "63ee615d62b9d001e6a2b660" # city
hua_result_id = "63ff8e895367c2261b4cb2ef"  # island

pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_loss_param", value_loss)
pop_dis.load_remote_input_dataset("housing_unit_allocation", hua_result_id)

pop_dis.set_input_dataset("building_dmg", combined_dmg)

result_name = "galveston-pop-disl-results"
seed = 1111

pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)
pop_dis.run_analysis()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [29]:
population_dislocation_result = pop_dis.get_output_dataset("result")
population_dislocation_result_df = population_dislocation_result.get_dataframe_from_csv(
    low_memory=False
)
population_dislocation_result_df.head()

,guid,DS_0,DS_1,DS_2,DS_3,haz_expose,huid,blockid,bgid,tractid,...,phispbg,d_sf,rploss_0,rploss_1,rploss_2,rploss_3,prdis,dislocated,choice_dis,unsafe_occ
0,3dab0b4b-42eb-4df6-a5e1-c1882df41704,1.000000,0.000000,0.000000,0.000000,no,B481677241012034H091,481677241012034,481677241012,48167724101,...,24.476440,0,0.002914,0.100425,0.524702,0.889881,0.000000,False,NaN,NaN
1,e474c4ef-a091-47ab-85b4-ef0459e37834,0.857675,0.095547,0.046773,0.000006,yes,B481677256003002H356,481677256003002,481677256003,48167725600,...,23.506289,0,0.006930,0.132025,0.677548,0.951019,0.297267,False,NaN,NaN
2,335d18da-7a60-4f4d-8c97-d35e0f3a1360,0.857567,0.095603,0.046824,0.000006,yes,B481677256003002H372,481677256003002,481677256003,48167725600,...,23.506289,0,0.005776,0.100000,0.654935,0.941974,0.294537,False,NaN,NaN
3,eb47b2e1-0f51-4e93-bd00-979cb6992b18,0.999373,0.000627,0.000000,0.000000,yes,B481677256003002H539,481677256003002,481677256003,48167725600,...,23.506289,0,0.002404,0.105851,0.436479,0.854591,0.270494,False,NaN,NaN
4,910074f1-f88e-4af3-ab70-15c6c62fb16d,0.063519,0.936455,0.000000,0.000026,yes,B481677259001031H113,481677259001031,481677259001,48167725900,...,19.912791,0,0.003029,0.133077,0.438777,0.855511,0.393400,True,NaN,NaN


## Housing Houshold Recovery Sequentials


### Social Vulnerabilities


In [30]:
social_vulnerability = SocialVulnerability(client)

social_vulnerability.set_parameter("result_name", "gal_social_vulnerabilty")
social_vulnerability.load_remote_input_dataset(
    "national_vulnerability_feature_averages", "63d178c2a011a9746c948115"
)
social_vulnerability.load_remote_input_dataset(
    "social_vulnerability_demographic_factors", "63d17e81a011a9746c94811b"
)
social_vulnerability.run_analysis()
sv_result = social_vulnerability.get_output_dataset("sv_result")

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


### HHRS


In [31]:
# Population dislocation result aggregated to the block group level.
# Transition probability matrix per social vulnerability level.
transition_probability_matrix = "60f5e2ae544e944c3cec0794"
# Initial mass probability function for stage 0.
initial_probability_vector = "60f5e918544e944c3cec668b"  # this is only for galveston???

# Create housing recovery instance
housing_recovery = HousingRecoverySequential(client)

# Load input datasets
housing_recovery.set_input_dataset(
    "population_dislocation_block", population_dislocation_result
)
housing_recovery.load_remote_input_dataset("tpm", transition_probability_matrix)
housing_recovery.load_remote_input_dataset(
    "initial_stage_probabilities", initial_probability_vector
)

# Chain with SV output
housing_recovery.set_input_dataset("sv_result", sv_result)

# Initial value to seed the random number generator to ensure replication
seed = 1111
# A size of the analysis time step.
t_delta = 1.0
# Total duration.
t_final = 90.0


# Specify the result name
result_name = "housing_recovery_result"

# Set analysis parameters
housing_recovery.set_parameter("result_name", result_name)
housing_recovery.set_parameter("seed", seed)
housing_recovery.set_parameter("t_delta", t_delta)
housing_recovery.set_parameter("t_final", t_final)

housing_recovery.run()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [32]:
HH_stagehistory_DF = housing_recovery.get_output_dataset(
    "ds_result"
).get_dataframe_from_csv()
timesteps = ["1", "7", "13", "25", "49", "85"]  # t0, t6, t12, t24, t48
HHRSOutputProcess.get_hhrs_stage_count(timesteps, HH_stagehistory_DF)

{'1': [6728, 167, 170, 20316, 0],
 '7': [1691, 2407, 1340, 21943, 0],
 '13': [1555, 2007, 1027, 22789, 3],
 '25': [1308, 1609, 659, 23720, 85],
 '49': [1010, 1061, 304, 24595, 411],
 '85': [85, 64, 9, 24888, 2335]}